# Pathway activity inference

## Loading packages

First, we need to load the relevant packages, `scanpy` to handle scRNA-seq data
and `decoupler` to use statistical methods.

In [ ]:
# %pip install decoupler
# %pip install omnipath
%pip install tabulate

In [ ]:
import scanpy as sc
import decoupler as dc

# Plotting options, change to your liking
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(4, 4)) 

## Loading the data

We can download the data easily using `scanpy`:

In [ ]:
adata = sc.datasets.pbmc3k_processed()
adata

We can visualize the different cell types in it:

In [ ]:
sc.pl.umap(adata, color='louvain')

## PROGENy model

[PROGENy](https://saezlab.github.io/progeny/) is a comprehensive resource containing a curated collection of pathways and their target genes, with weights for each interaction.
For this example we will use the human weights (other organisms are available) and we will use the top 500 responsive genes ranked by p-value. Here is a brief description of each pathway:

- **Androgen**: involved in the growth and development of the male reproductive organs.
- **EGFR**: regulates growth, survival, migration, apoptosis, proliferation, and differentiation in mammalian cells
- **Estrogen**: promotes the growth and development of the female reproductive organs.
- **Hypoxia**: promotes angiogenesis and metabolic reprogramming when O2 levels are low.
- **JAK-STAT**: involved in immunity, cell division, cell death, and tumor formation.
- **MAPK**: integrates external signals and promotes cell growth and proliferation.
- **NFkB**: regulates immune response, cytokine production and cell survival.
- **p53**: regulates cell cycle, apoptosis, DNA repair and tumor suppression.
- **PI3K**: promotes growth and proliferation.
- **TGFb**: involved in development, homeostasis, and repair of most tissues.
- **TNFa**: mediates haematopoiesis, immune surveillance, tumour regression and protection from infection.
- **Trail**: induces apoptosis.
- **VEGF**: mediates angiogenesis, vascular permeability, and cell migration.
- **WNT**: regulates organ morphogenesis during development and tissue repair.

To access it we can use `decoupler`.

In [ ]:
progeny = dc.get_progeny(organism='human', top=500)
progeny

## Activity inference with multivariate linear model (MLM)

To infer pathway enrichment scores we will run the multivariate linear model (`mlm`) method. For each cell in our dataset (`adata`), it fits a linear model that predicts the observed gene expression based on all pathways' Pathway-Gene interactions weights.
Once fitted, the obtained t-values of the slopes are the scores. If it is positive, we interpret that the pathway is active and if it is negative we interpret that it is inactive.

<img src="../mlm.png" />

To run `decoupler` methods, we need an input matrix (`mat`), an input prior knowledge
network/resource (`net`), and the name of the columns of `net` that we want to use.

In [ ]:
dc.run_mlm(
    mat=adata,
    net=progeny,
    source='source',
    target='target',
    weight='weight',
    verbose=True
)

The obtained scores (t-values)(`mlm_estimate`) and p-values (`mlm_pvals`) are stored in the `.obsm` key:

In [ ]:
adata.obsm['mlm_estimate']

**Note**: Each run of `run_mlm` overwrites what is inside of `mlm_estimate` and `mlm_pvals`. if you want to run `mlm` with other resources and still keep the activities inside the same `AnnData` object, you can store the results in any other key in `.obsm` with different names, for example:

In [ ]:
adata.obsm['progeny_mlm_estimate'] = adata.obsm['mlm_estimate'].copy()
adata.obsm['progeny_mlm_pvals'] = adata.obsm['mlm_pvals'].copy()
adata

## Visualization

To visualize the obtained scores, we can re-use many of `scanpy`'s plotting functions.
First though, we need to extract the activities from the `adata` object.

In [ ]:
acts = dc.get_acts(adata, obsm_key='mlm_estimate')
acts

`dc.get_acts` returns a new `AnnData` object which holds the obtained activities in its `.X` attribute, allowing us to re-use many `scanpy` functions, for example let's visualise the Trail pathway:

In [ ]:
sc.pl.umap(acts, color=['Trail', 'louvain'], cmap='RdBu_r', vcenter=0)
sc.pl.violin(acts, keys=['Trail'], groupby='louvain', rotation=90)

It seem that in B cells, the pathway Trail, associated with apoptosis, is more active.

## Exploration

We can visualize which pathways are more active in each cell type:

In [ ]:
sc.pl.matrixplot(acts, var_names=acts.var_names, groupby='louvain', dendrogram=True, standard_scale='var',
                 colorbar_title='Z-scaled scores', cmap='RdBu_r')

In this specific example, we can observe that EGFR to be more active in Megakaryocytes, and that Trail is more active in B cells.

Finally, we can check individual pathways by plotting their distributions:

In [ ]:
sc.pl.violin(acts, keys=['EGFR'], groupby='louvain', rotation=90)

In [ ]:
acts.obsm['mlm_estimate'].head()

In [ ]:
acts.obsm['mlm_pvals'].head()

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Extract the data matrix for the selected variables
data = acts[:, acts.var_names].X  # Use acts.var_names for all variables

# Standardize the data (Z-scaling across variables)
scaler = StandardScaler()
z_scaled_data = scaler.fit_transform(data)

# Convert to a DataFrame for easier manipulation
z_scaled_df = pd.DataFrame(
    z_scaled_data, 
    index=acts.obs.index, 
    columns=acts.var_names
)

# Add the 'louvain' column to group by cell type
z_scaled_df['louvain'] = acts.obs['louvain']

# Group by 'louvain' and compute the mean Z-scaled scores
z_scaled_grouped = z_scaled_df.groupby('louvain').mean()

# Display the Z-scaled scores
print(z_scaled_grouped)

In [ ]:
from tabulate import tabulate

# Convert the DataFrame to a Markdown table
markdown_table = tabulate(z_scaled_grouped, headers='keys', tablefmt='pipe')

# Print the Markdown table
print(markdown_table)